In [1]:
%matplotlib inline

## Datasets Preparation

- Create Balanced Data set from the good quality images of the 1.25x 
- This dataset will have 200 images for each class.
- the remain images will be moved to the Val and Test datasets.
- The split between the val - test will be 50%-50%.
- I want to measure the performance of balanced dataset compared to the original which received good score

In [2]:
# import YOLO model
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import os

## Comparison between Different Yolo Models
Currently only YOLO v8 and v11 support image classification
- YOLO v8
- YOLO v11


In [5]:
import os

yolo_pre_trained_folder = "/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/pre_trained"

yolo_classification_checkpoints = {
    "YOLOv8": {
        "Nano": os.path.join(yolo_pre_trained_folder, "yolov8n-cls.pt"),
        "Small": os.path.join(yolo_pre_trained_folder, "yolov8s-cls.pt"),
        "Medium": os.path.join(yolo_pre_trained_folder, "yolov8m-cls.pt"),
        "Large": os.path.join(yolo_pre_trained_folder, "yolov8l-cls.pt"),
        "XLarge": os.path.join(yolo_pre_trained_folder, "yolov8x-cls.pt"),
    },
        "YOLOv11": {
        "Nano": os.path.join(yolo_pre_trained_folder, "yolo11n-cls.pt"),
        "Small": os.path.join(yolo_pre_trained_folder, "yolo11s-cls.pt"),
        "Medium": os.path.join(yolo_pre_trained_folder, "yolo11m-cls.pt"),
        "Large": os.path.join(yolo_pre_trained_folder, "yolo11l-cls.pt"),
        "XLarge": os.path.join(yolo_pre_trained_folder, "yolo11x-cls.pt"),
    },
}

Functions to fine tune each model and then evaluate it

In [6]:
def train_model(data_path, model_name, model_size, epochs=10, imgsz=128):
    # Load the model checkpoint
    checkpoint = yolo_classification_checkpoints[model_name][model_size]
    model = YOLO(checkpoint)
    
    # Train the model
    model.train(data=data_path, epochs=epochs, imgsz=imgsz)
    
    return model

def eval_model(model, test_data_path):
    # Get class names from the test dataset directory
    class_names = sorted(os.listdir(test_data_path))
    class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}

    # Initialize lists for true labels (y_test) and predicted labels (y_pred)
    y_test = []
    y_pred = []

    # Initialize a list to store image data for plotting
    images_for_plotting = []

    # Process each class in the test directory
    for class_name in class_names:
        class_dir = os.path.join(test_data_path, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            
            # Load the image and run the model prediction
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib
            results = model(img)
            
            # Get the predicted class index
            predicted_idx = int(results[0].probs.data.argmax())
            
            # Append true and predicted labels
            y_test.append(class_to_idx[class_name])
            y_pred.append(predicted_idx)
            
            # Store the image, true label, and predicted label for plotting
            images_for_plotting.append((img_rgb, class_name, class_names[predicted_idx]))

    # Evaluate using classification metrics
    classification_report_result = classification_report(y_test, y_pred, target_names=class_names)
    accuracy = accuracy_score(y_test, y_pred)
    confusion_matrix_result = confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return classification_report_result, accuracy, confusion_matrix_result, precision, recall, f1, images_for_plotting
    
def plot_results(images_for_plotting):
    # Plot images with true and predicted labels
    plt.figure(figsize=(12, 12))
    num_images = len(images_for_plotting)  # Display all images (adjust as needed)
    rows = (num_images // 4) + 1  # Adjust rows based on the number of images

    for i in range(num_images):
        img, true_label, pred_label = images_for_plotting[i]
        plt.subplot(rows, 4, i + 1)  # Dynamic row setting
        plt.imshow(img)
        plt.title(f"True: {true_label}\nPred: {pred_label}", color="green" if true_label == pred_label else "red")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

Train models on multiple datasets for cross validation

In [7]:
import os

# iterate over each dataset in the datasets folder
balanced_datasets = "/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/balanced_datasets"
data_paths = [os.path.join(balanced_datasets, dataset) for dataset in os.listdir(balanced_datasets)]
fine_tuned_checkpoints = "/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/fine_tuned"

results = []  # List to store the evaluation results

for i, data_path in enumerate(data_paths):
    test_data_path = os.path.join(data_path, "val")
    # run the functions for each of the models
    for model_name in yolo_classification_checkpoints:
        for model_size in yolo_classification_checkpoints[model_name]:
            print(f"Training {model_name} {model_size} model...")
            model = train_model(data_path, model_name, model_size, epochs=10, imgsz=128)
            
            print(f"Evaluating {model_name} {model_size} model...")
            classification_report_result, accuracy, confusion_matrix_result, precision, recall, f1, images_for_plotting = eval_model(model, test_data_path)
            
            # create saving folder if does not exist
            if not os.path.exists(f"{fine_tuned_checkpoints}/{model_name}"):
                os.makedirs(f"{fine_tuned_checkpoints}/{model_name}")
            
            # save model
            model.save(f"{fine_tuned_checkpoints}/{model_name}/{model_size}_dataset_{i}.pt")
            
            # Append the evaluation results to the list
            results.append({
                "Model_Name": model_name,
                "Model_Size": model_size,
                "Classification Report": classification_report_result,
                "Accuracy": accuracy,
                "Average Precision": precision,
                "Average Recall": recall,
                "Average f1": f1,
                "Confusion Matrix": confusion_matrix_result,
                "Images for Plotting": images_for_plotting
            })

Training YOLOv8 Nano model...
Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA TITAN X (Pascal), 12183MiB)
engine/trainer: task=classify, mode=train, model=/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/pre_trained/yolov8n-cls.pt, data=/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/balanced_datasets/balanced_2, epochs=10, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=F

train: Scanning /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/balanced_datasets/balanced_2/train... 462 images, 0 corrupt: 100%|██████████| 462/462 [00:00<00:00, 1113.71it/s]


train: New cache created: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/balanced_datasets/balanced_2/train.cache


/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/balanced_datasets/balanced_2/val... 156 images, 0 corrupt: 100%|██████████| 156/156 [00:00<00:00, 931.75it/s]

val: New cache created: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/balanced_datasets/balanced_2/val.cache


...

      10/10       1.6G      0.221         14        128: 100%|██████████| 29/29 [00:04<00:00,  6.21it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 20.88it/s]

                   all      0.981          1



10 epochs completed in 0.050 hours.
Optimizer stripped from /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/classify/train50/weights/last.pt, 57.0MB
Optimizer stripped from /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/classify/train50/weights/best.pt, 57.0MB

Validating /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/classify/train50/weights/best.pt...
Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA TITAN X (Pascal), 12183MiB)
YOLO11x-cls summary (fused): 227 layers, 28,336,259 parameters, 0 gradients, 110.3 GFLOPs
train: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/balanced_datasets/balanced_5/train... found 462 images in 3 classes ✅ 
val: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/balanced_datasets/balanced_5/val... found 156 images in 3 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00,  7.90it/s]


                   all      0.981          1
Speed: 0.0ms preprocess, 2.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/classify/train50
Evaluating YOLOv11 XLarge model...

0: 128x128 amber 1.00, cloudy 0.00, clear 0.00, 11.7ms
Speed: 3.1ms preprocess, 11.7ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 cloudy 0.55, amber 0.44, clear 0.01, 14.2ms
Speed: 2.5ms preprocess, 14.2ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 1.00, cloudy 0.00, clear 0.00, 13.5ms
Speed: 2.2ms preprocess, 13.5ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 0.89, cloudy 0.10, clear 0.01, 14.6ms
Speed: 2.3ms preprocess, 14.6ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 1.00, cloudy 0.00, clear 0.00, 10.0ms
Speed: 1.7ms preprocess, 10.0ms inference, 0.1ms postpro

Train Models on a single dataset

Split the data by ratio for train and val

In [4]:
import os
import random
import shutil

def split_images_by_ratio(
    source_class_path,
    target_train_path,
    target_test_path,
    train_ratio=0.75
):
    """
    Splits images from `source_class_path` into train and test folders, 
    using `train_ratio` proportion for training and (1 - train_ratio) for testing.
    This split is done PER CLASS folder (so each class has the same ratio).

    :param source_class_path: Path to the folder containing images for a single class
    :param target_train_path: Path to the 'train/class_name' folder for the training images
    :param target_test_path:  Path to the 'test/class_name' folder for the testing images
    :param train_ratio:       Proportion of images to use for the training set (0 to 1).
    """

    # Get all files (only actual files) in the class folder.
    images = [
        f for f in os.listdir(source_class_path) 
        if os.path.isfile(os.path.join(source_class_path, f))
    ]

    # Shuffle the list of images
    random.shuffle(images)

    # Compute the split index for train vs test
    split_index = int(len(images) * train_ratio)

    # Train images are up to the split_index, test images are after that
    train_images = images[:split_index]
    test_images  = images[split_index:]

    # Copy images to the train directory
    for i, img_name in enumerate(train_images):
        src_img_path  = os.path.join(source_class_path, img_name)
        # We can prepend an index to avoid filename collisions
        dest_img_path = os.path.join(target_train_path, f"{i}_{img_name}")
        shutil.copy2(src_img_path, dest_img_path)

    # Copy images to the test directory
    for i, img_name in enumerate(test_images):
        src_img_path  = os.path.join(source_class_path, img_name)
        dest_img_path = os.path.join(target_test_path, f"{i}_{img_name}")
        shutil.copy2(src_img_path, dest_img_path)


def create_train_test_split_by_ratio(
    dataset_path,
    split_dataset_path,
    train_ratio=0.8
):
    """
    Iterates through each class folder in `dataset_path` and splits images
    into train/test subfolders under `split_dataset_path`, 
    respecting the chosen train:test ratio for each class independently.

    :param dataset_path:       Path to the folder that contains subfolders for each class
    :param split_dataset_path: Path to the folder where the train/test split will be created
    :param train_ratio:        Proportion of images that will go into the train set (0 to 1)
    """

    # Create the output folder if it doesn't exist
    os.makedirs(split_dataset_path, exist_ok=True)

    # The new train/test dirs:
    train_root = os.path.join(split_dataset_path, "train")
    test_root  = os.path.join(split_dataset_path, "val")
    os.makedirs(train_root, exist_ok=True)
    os.makedirs(test_root, exist_ok=True)

    # Iterate over each class folder in the dataset
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)

        # Skip if it's not a directory, or if it's 'train'/'test' already in the source
        if not os.path.isdir(class_path):
            continue
        if class_name.lower() in ["train", "val"]:
            # Skip if you already have train/test in the original folder
            # (Remove this check if you want to handle them differently)
            continue

        # Create class subfolders in the new train and test directories
        target_train_path = os.path.join(train_root, class_name)
        target_test_path  = os.path.join(test_root,  class_name)
        os.makedirs(target_train_path, exist_ok=True)
        os.makedirs(target_test_path,  exist_ok=True)

        print(f"Splitting class '{class_name}' images with ratio {train_ratio*100:.0f}% / {100 - train_ratio*100:.0f}% ...")

        # Perform the ratio-based split for this class
        split_images_by_ratio(
            source_class_path=class_path,
            target_train_path=target_train_path,
            target_test_path=target_test_path,
            train_ratio=train_ratio
        )

    print("\nDone! Train/test split created at:", split_dataset_path)


def print_dataset_stats(split_dataset_path):
    """
    Prints the number of images in each class under train/ and test/ folders,
    and shows totals for train and test sets.
    """
    train_path = os.path.join(split_dataset_path, "train")
    test_path = os.path.join(split_dataset_path, "val")

    # Check if train/test directories exist
    if not os.path.isdir(train_path):
        print(f"Train directory does not exist: {train_path}")
        return
    if not os.path.isdir(test_path):
        print(f"Test directory does not exist: {test_path}")
        return

    print("===== TRAIN FOLDER STATS =====")
    train_total = 0
    for class_name in os.listdir(train_path):
        class_dir = os.path.join(train_path, class_name)
        if not os.path.isdir(class_dir):
            continue

        # Count how many files are in this class directory
        images = [
            f for f in os.listdir(class_dir)
            if os.path.isfile(os.path.join(class_dir, f))
        ]
        n_images = len(images)
        train_total += n_images
        print(f"  Class '{class_name}': {n_images} images")

    print(f"Total images in train folder: {train_total}\n")

    print("===== TEST FOLDER STATS =====")
    test_total = 0
    for class_name in os.listdir(test_path):
        class_dir = os.path.join(test_path, class_name)
        if not os.path.isdir(class_dir):
            continue

        images = [
            f for f in os.listdir(class_dir)
            if os.path.isfile(os.path.join(class_dir, f))
        ]
        n_images = len(images)
        test_total += n_images
        print(f"  Class '{class_name}': {n_images} images")

    print(f"Total images in test folder: {test_total}")

In [5]:
dataset_path = "/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/good_quality"
balanced_dataset = "/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality"
train_ratio = 0.75
# Adjust target_count as needed
create_train_test_split_by_ratio(
    dataset_path=dataset_path,
    split_dataset_path=balanced_dataset,
    train_ratio=train_ratio
)

Splitting class 'amber' images with ratio 75% / 25% ...
Splitting class 'clear' images with ratio 75% / 25% ...
Splitting class 'cloudy' images with ratio 75% / 25% ...

Done! Train/test split created at: /home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality


In [6]:
print_dataset_stats(balanced_dataset)

===== TRAIN FOLDER STATS =====
  Class 'amber': 155 images
  Class 'clear': 153 images
  Class 'cloudy': 154 images
Total images in train folder: 462

===== TEST FOLDER STATS =====
  Class 'amber': 52 images
  Class 'clear': 52 images
  Class 'cloudy': 52 images
Total images in test folder: 156


In [9]:
import os

# iterate over each dataset in the datasets folder
dataset_path = "/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality"
train_path = os.path.join(dataset_path, "train")
test_path = os.path.join(dataset_path, "val")

results = []  # List to store the evaluation results

# run the functions for each of the models
for model_name in yolo_classification_checkpoints:
    for model_size in yolo_classification_checkpoints[model_name]:
        print(f"Training {model_name} {model_size} model...")
        model = train_model(dataset_path, model_name, model_size, epochs=10, imgsz=128)
        
        print(f"Evaluating {model_name} {model_size} model...")
        classification_report_result, accuracy, confusion_matrix_result, precision, recall, f1, images_for_plotting = eval_model(model, test_path)
        
        # create saving folder if does not exist
        if not os.path.exists(f"/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/balanced_good_quality/{model_name}"):
            os.makedirs(f"/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/balanced_good_quality/{model_name}")
        
        # save model
        model.save(f"/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/balanced_good_quality/{model_name}/{model_size}.pt")
        
        # Append the evaluation results to the list
        results.append({
            "Model_Name": model_name,
            "Model_Size": model_size,
            "Classification Report": classification_report_result,
            "Accuracy": accuracy,
            "Average Precision": precision,
            "Average Recall": recall,
            "Average f1": f1,
            "Confusion Matrix": confusion_matrix_result,
            "Images for Plotting": images_for_plotting
        })

Training YOLOv8 Nano model...
Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24253MiB)
engine/trainer: task=classify, mode=train, model=/home/etaylor/code_projects/thesis/checkpoints/trichome_image_classification/yolo/yolov8n-cls.pt, data=/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality, epochs=10, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train333, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stre

train: Scanning /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality/train... 462 images, 0 corrupt: 100%|██████████| 462/462 [00:00<00:00, 1832.56it/s]

train: New cache created: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality/train.cache



/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality/val... 156 images, 0 corrupt: 100%|██████████| 156/156 [00:00<00:00, 1440.48it/s]

val: New cache created: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality/val.cache


...

      10/10      1.87G     0.1619         14        128: 100%|██████████| 29/29 [00:01<00:00, 17.03it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 58.01it/s]

                   all      0.962          1



10 epochs completed in 0.031 hours.
Optimizer stripped from /home/etaylor/runs/classify/train342/weights/last.pt, 57.0MB
Optimizer stripped from /home/etaylor/runs/classify/train342/weights/best.pt, 57.0MB

Validating /home/etaylor/runs/classify/train342/weights/best.pt...
Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24253MiB)
YOLO11x-cls summary (fused): 227 layers, 28,336,259 parameters, 0 gradients, 110.3 GFLOPs
train: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality/train... found 462 images in 3 classes ✅ 
val: /sise/home/etaylor/code_projects/thesis/classification_datasets/trichome_classification/extracted_trichomes_images/balanced_good_quality/val... found 156 images in 3 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 25.96it/s]


                   all      0.968          1
Speed: 0.0ms preprocess, 1.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /home/etaylor/runs/classify/train342
Evaluating YOLOv11 XLarge model...

0: 128x128 amber 1.00, clear 0.00, cloudy 0.00, 8.7ms
Speed: 1.4ms preprocess, 8.7ms inference, 0.1ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 0.87, cloudy 0.09, clear 0.05, 8.9ms
Speed: 1.3ms preprocess, 8.9ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 1.00, clear 0.00, cloudy 0.00, 8.8ms
Speed: 1.4ms preprocess, 8.8ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 1.00, cloudy 0.00, clear 0.00, 8.9ms
Speed: 1.3ms preprocess, 8.9ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 0.97, cloudy 0.03, clear 0.00, 8.9ms
Speed: 1.3ms preprocess, 8.9ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 amber 1.00, clear

In [8]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df

,Model_Name,Model_Size,Classification Report,Accuracy,Average Precision,Average Recall,Average f1,Confusion Matrix,Images for Plotting
0,YOLOv8,Nano,precision recall f1-score ...,0.987179,0.987300,0.987179,0.987179,"[[52, 0, 0], [1, 51, 0], [0, 1, 51]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
1,YOLOv8,Small,precision recall f1-score ...,0.993590,0.993711,0.993590,0.993589,"[[52, 0, 0], [0, 51, 1], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
2,YOLOv8,Medium,precision recall f1-score ...,0.987179,0.987300,0.987179,0.987179,"[[51, 1, 0], [0, 51, 1], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
3,YOLOv8,Large,precision recall f1-score ...,0.980769,0.981818,0.980769,0.980753,"[[52, 0, 0], [0, 49, 3], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
4,YOLOv8,XLarge,precision recall f1-score ...,0.980769,0.981365,0.980769,0.980635,"[[52, 0, 0], [2, 49, 1], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
5,YOLOv11,Nano,precision recall f1-score ...,0.974359,0.974577,0.974359,0.974229,"[[52, 0, 0], [2, 49, 1], [0, 1, 51]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
6,YOLOv11,Small,precision recall f1-score ...,0.987179,0.987421,0.987179,0.987115,"[[52, 0, 0], [1, 50, 1], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
7,YOLOv11,Medium,precision recall f1-score ...,0.987179,0.987421,0.987179,0.987115,"[[52, 0, 0], [1, 50, 1], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
8,YOLOv11,Large,precision recall f1-score ...,0.980769,0.981365,0.980769,0.980635,"[[49, 1, 2], [0, 52, 0], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
9,YOLOv11,XLarge,precision recall f1-score ...,0.987179,0.987300,0.987179,0.987179,"[[51, 1, 0], [0, 51, 1], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...


In [9]:
# sort the model results based on the f1 score
results_df = results_df.sort_values(by="Average f1", ascending=False)
results_df

,Model_Name,Model_Size,Classification Report,Accuracy,Average Precision,Average Recall,Average f1,Confusion Matrix,Images for Plotting
44,YOLOv8,XLarge,precision recall f1-score ...,1.000000,1.000000,1.000000,1.000000,"[[52, 0, 0], [0, 52, 0], [0, 0, 52]]",[([[[ 91 91 67]\n [108 108 84]\n [126 125 1...
1,YOLOv8,Small,precision recall f1-score ...,0.993590,0.993711,0.993590,0.993589,"[[52, 0, 0], [0, 51, 1], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
17,YOLOv11,Medium,precision recall f1-score ...,0.990291,0.990566,0.990291,0.990289,"[[103, 0, 0], [0, 100, 3], [0, 0, 103]]",[([[[199 202 185]\n [199 202 185]\n [200 203 1...
13,YOLOv8,Large,precision recall f1-score ...,0.990291,0.990566,0.990291,0.990289,"[[103, 0, 0], [0, 103, 0], [0, 3, 100]]",[([[[199 202 185]\n [199 202 185]\n [200 203 1...
11,YOLOv8,Small,precision recall f1-score ...,0.990291,0.990566,0.990291,0.990289,"[[103, 0, 0], [0, 103, 0], [0, 3, 100]]",[([[[199 202 185]\n [199 202 185]\n [200 203 1...
36,YOLOv11,Small,precision recall f1-score ...,0.987179,0.987654,0.987179,0.987238,"[[51, 0, 1], [0, 51, 1], [0, 0, 52]]",[([[[ 87 97 63]\n [ 89 99 65]\n [ 91 101 ...
9,YOLOv11,XLarge,precision recall f1-score ...,0.987179,0.987300,0.987179,0.987179,"[[51, 1, 0], [0, 51, 1], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
2,YOLOv8,Medium,precision recall f1-score ...,0.987179,0.987300,0.987179,0.987179,"[[51, 1, 0], [0, 51, 1], [0, 0, 52]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
0,YOLOv8,Nano,precision recall f1-score ...,0.987179,0.987300,0.987179,0.987179,"[[52, 0, 0], [1, 51, 0], [0, 1, 51]]",[([[[ 66 81 52]\n [ 65 80 51]\n [ 68 83 ...
21,YOLOv8,Small,precision recall f1-score ...,0.987179,0.987300,0.987179,0.987179,"[[52, 0, 0], [1, 51, 0], [0, 1, 51]]",[([[[ 95 108 91]\n [ 85 98 81]\n [ 79 92 ...


In [10]:
# create an average result dataframe for each of the models
average_results = []

for model_name in yolo_classification_checkpoints:
    for model_size in yolo_classification_checkpoints[model_name]:
        model_results = results_df[(results_df["Model_Name"] == model_name) & (results_df["Model_Size"] == model_size)]
        
        # Calculate the average accuracy
        avg_accuracy = model_results["Accuracy"].mean()
        avg_precision = model_results["Average Precision"].mean()
        avg_recall = model_results["Average Recall"].mean()
        avg_f1 = model_results["Average f1"].mean()
        
        # Calculate the average confusion matrix
        avg_confusion_matrix = model_results["Confusion Matrix"].mean(axis=0)
        
        # Append the average results to the list
        average_results.append({
            "Model Name": model_name,
            "Model Size": model_size,
            "Average Accuracy": avg_accuracy,
            "Average Precision": avg_precision,
            "Average Recall": avg_recall,
            "Average f1": avg_f1,
            "Average Confusion Matrix": avg_confusion_matrix
        })

In [11]:
average_results_df = pd.DataFrame(average_results)

# sort the average results based on the f1 score
average_results_df = average_results_df.sort_values(by="Average f1", ascending=False)

average_results_df


,Model Name,Model Size,Average Accuracy,Average Precision,Average Recall,Average f1,Average Confusion Matrix
4,YOLOv8,XLarge,0.986507,0.986937,0.986507,0.986476,"[[62.2, 0.0, 0.0], [0.4, 60.8, 1.0], [0.0, 1.2..."
1,YOLOv8,Small,0.985238,0.985389,0.985238,0.985237,"[[62.0, 0.2, 0.0], [0.2, 61.2, 0.8], [0.2, 1.2..."
9,YOLOv11,XLarge,0.983308,0.983599,0.983308,0.983331,"[[61.6, 0.2, 0.4], [0.0, 60.4, 1.8], [0.2, 0.4..."
3,YOLOv8,Large,0.982674,0.983197,0.982674,0.982705,"[[61.6, 0.2, 0.4], [0.0, 60.8, 1.4], [0.2, 0.8..."
6,YOLOv11,Small,0.982026,0.982373,0.982026,0.982009,"[[61.8, 0.0, 0.4], [0.6, 60.6, 1.0], [0.2, 1.0..."
2,YOLOv8,Medium,0.980744,0.980911,0.980744,0.980730,"[[61.8, 0.4, 0.0], [0.2, 60.2, 1.8], [0.4, 0.6..."
8,YOLOv11,Large,0.980744,0.981241,0.980744,0.980673,"[[61.6, 0.2, 0.4], [0.0, 60.6, 1.6], [0.2, 1.0..."
7,YOLOv11,Medium,0.977545,0.978323,0.977545,0.977594,"[[61.2, 0.4, 0.6], [0.2, 60.6, 1.4], [0.0, 1.2..."
0,YOLOv8,Nano,0.977508,0.977637,0.977508,0.977467,"[[62.0, 0.0, 0.2], [0.6, 60.0, 1.6], [0.4, 1.6..."
5,YOLOv11,Nano,0.976251,0.976555,0.976251,0.976209,"[[62.0, 0.0, 0.2], [0.4, 60.2, 1.6], [0.4, 1.6..."


In [33]:
# # save the results to a file
# average_results_df.to_csv("yolo_classification_avg_results_for_5_datasets.csv", index=False)
# results_df.to_csv("yolo_classification_results_for_5_datasets.csv", index=False)

In [36]:
print(average_results_df.columns)

Index(['Model Name', 'Model Size', 'Average Accuracy', 'Average Precision',
       'Average Recall', 'Average f1', 'Average Confusion Matrix'],
      dtype='object')


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Convert the average results to a DataFrame
average_results_df = pd.DataFrame(average_results)

# Plot the key metrics (accuracy, precision, recall, F1) across different models and sizes
metrics = ["Average Accuracy", "Average Precision", "Average Recall", "Average f1"]

plt.figure(figsize=(16, 12))

for i, metric in enumerate(metrics, 1):
    plt.subplot(2, 2, i)
    sns.barplot(data=average_results_df, x="Model Size", y=metric, hue="Model Name")
    plt.title(f"Comparison of {metric} Across Models and Sizes")
    plt.ylabel(metric)
    plt.xlabel("Model Size")
    plt.legend(title="Model Name")

plt.tight_layout()
plt.show()


<Figure size 1600x1200 with 4 Axes>

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define the number of rows and columns for the grid
num_matrices = len(average_results_df)
ncols = 3  # Number of columns in the grid
nrows = (num_matrices // ncols) + (num_matrices % ncols > 0)  # Calculate rows needed

# Set up the figure for the grid of confusion matrices
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 5))

# Flatten axes to easily iterate over them
axes = axes.flatten()

# Plot each confusion matrix in the grid
for index, (ax, row) in enumerate(zip(axes, average_results_df.iterrows())):
    row_data = row[1]
    sns.heatmap(row_data["Average Confusion Matrix"], annot=True, fmt=".2f", cmap="Blues", cbar=True, ax=ax)
    ax.set_title(f"{row_data['Model Name']} - {row_data['Model Size']}")
    ax.set_xlabel("Predicted Class")
    ax.set_ylabel("True Class")

# Remove any unused subplots
for i in range(num_matrices, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()


<Figure size 1500x2000 with 20 Axes>

## Experiment Settings
- 200 Images for each class for training
- The rest of the images goes to val and test sets
- Val and test sets splits are 50%-50% (26 - clear, 150 - cloudy and 89 - amber )

## Experiment results
- Good Results
- Around 93% for all metrics (accuracy, precision, recall and f1)
- In conf matrix class 0 = amber, 1 = clear and 2 = cloudy
- Best model to predict amber class is yolov8 Large.
